In [92]:
import sys
import os

# خلي Python يشوف مجلد src
sys.path.append(os.path.abspath(os.path.join("..", "src")))
# دلوقتي تقدر تستدعي الموديولز
from features.distances import add_distance_features
from features.timeparts import add_time_features
from features.encoding import add_one_hot_encoding
from data.Data_helper import save_processed_data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [93]:
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")

print(df.shape)
print(df.head())
print(df.info())

(1000000, 10)
          id  vendor_id      pickup_datetime  passenger_count  \
0  id2793718          2  2016-06-08 07:36:19                1   
1  id3485529          2  2016-04-03 12:58:11                1   
2  id1816614          2  2016-06-05 02:49:13                5   
3  id1050851          2  2016-05-05 17:18:27                2   
4  id0140657          1  2016-05-12 17:43:38                4   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.985611        40.735943         -73.980331         40.760468   
1        -73.978394        40.764351         -73.991623         40.749859   
2        -73.989059        40.744389         -73.973381         40.748692   
3        -73.990326        40.731136         -73.991264         40.748917   
4        -73.789497        40.646675         -73.987137         40.759232   

  store_and_fwd_flag  trip_duration  
0                  N           1040  
1                  N            827  
2                 

دلوقتي خدنا فكره عامه عن الداتا بتاعتنا لقيناها مش معبره كفايه ولكن منها نقدر ننشأ features جديده تكون معبره اكتر وليها علاقه مباشره بالتارجت بتاعنا 
زي ايه بقا نجيب بعض المسافات من معلومات خطوط االطول ودواير العرض بتاع مكان الاقلاع ومكان الانتهاء 
Distances: Haversine, Manhattan
Directions: Bearing ,speed_kmh
وبرضو عمود الداتاتايم لو فصلناه يبقا افضل لان الساعه ممكن تكون اهم من التاريخ وهل اليوم ده كان ويك اند ولا عادي فبرضو هنعمل شغل علي اعمدة الداتا تايم 
كل ده هنستخدم ملف  ال features لان كل الدوال منظمه جواه

In [94]:
# ==== 9️⃣ Feature Ideas Generation ====

# 1. Distance-based features
from features.distances import add_distance_features
from features.timeparts import add_time_features

# Make sure df is defined
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")
df['trip_duration_min'] = df['trip_duration'] / 60
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

df = add_distance_features(df)

# 2. Time-based features (using pickup_datetime)
df = add_time_features(df, datetime_col="pickup_datetime")

print("\n Columns after add_distance_features:")
print(df.columns.tolist())

# 3. Quick check
print("\n✅ Features added successfully!")
print(df.info())
print(df.head(10)[[
    "trip_duration_min", 
    "haversine_km", "manhattan_km", "bearing_deg",
    "hour", "weekday", "month", "is_weekend", "is_rush_hour"
]])
print(df[["trip_duration_min", "haversine_km", "manhattan_km", "speed_kmh", "bearing_deg"]].head())

✅ add_distance_features: 1000000/1000000 rows got valid speed_kmh

 Columns after add_distance_features:
['id', 'vendor_id', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'weekday', 'month', 'is_weekend', 'is_rush_hour']

✅ Features added successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1000000 non-null  object        
 1   vendor_id           1000000 non-null  int64         
 2   pickup_datetime     1000000 non-null  datetime64[ns]
 3   passenger_count     1000000 non-null  int64         
 4   pickup_longitude    1000000 non-null  float64       
 5   pickup_latitude     1000

دلوقتي بعد ما ضيفنا الفيتشرز الجديده وشوفناا شويه منهم فعلا نقدر نقول انها منطقيه اكتر وليها معني 
المسافات الاكبر بتعمل وقت اعلي ولو مسافتين متساويتين السرعه الاعلي تعمل مده اقل 

الداتا عندنا بتاع التارجت فيها شوية  skewed 
Log Transformationيعني التوزيع بتاعها مش طبيعي اوي عشان كده رأيي نطبق عليه دالة ال 

In [95]:
df = df.drop(columns=["trip_duration"])


In [96]:
df["log_trip_duration_min"] = np.log1p(df["trip_duration_min"])
print (df.head(10))

          id  vendor_id     pickup_datetime  passenger_count  \
0  id2793718          2 2016-06-08 07:36:19                1   
1  id3485529          2 2016-04-03 12:58:11                1   
2  id1816614          2 2016-06-05 02:49:13                5   
3  id1050851          2 2016-05-05 17:18:27                2   
4  id0140657          1 2016-05-12 17:43:38                4   
5  id2734699          2 2016-04-14 12:17:42                3   
6  id2151697          2 2016-03-17 21:10:43                1   
7  id3635863          1 2016-02-05 06:43:21                1   
8  id2169697          2 2016-03-22 14:08:02                1   
9  id2225613          1 2016-03-29 23:32:47                1   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.985611        40.735943         -73.980331         40.760468   
1        -73.978394        40.764351         -73.991623         40.749859   
2        -73.989059        40.744389         -73.973381         

عندنا أعمدة في الداتا مش أرقام، او ارقام مترتبه
:
- `vendor_id`
- `store_and_fwd_flag`
- `hour`
- `weekday`
- `month`

الموديل ما بيعرفش يتعامل مع الحروف أو القيم اللي مش أرقام.  
ولو أرقام عادية (1، 2، 3 …) الموديل هيفتكر إن في ترتيب ومعنى للفرق بينهم، وده مش صح.

عشان كده بنستخدم **One-Hot Encoding**:
- بيحوّل كل قيمة كاتيجوري لعمود جديد فيه 0 و 1.
- بيمنع الموديل يفتكر إن في علاقة ترتيب بين القيم.
- بيسهّل على الموديل يتعلم صح.
- 
مثال:  
لو عندنا `vendor_id` (1 أو 2) → بعد الـ encoding هيتحوّل لعمود `vendor_id_2` فيه:
- 1 لو الفيندور = 2
- 0 لو الفيندور = 1  

وبكده نكون مثلنا الكاتيجوري بطريقة أوضح للموديل.

In [97]:
df = add_one_hot_encoding(df)
print(df.columns.tolist())

print(df.head(10))

['id', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'is_weekend', 'is_rush_hour', 'log_trip_duration_min', 'vendor_id_1', 'vendor_id_2', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6']
          id     pickup_datetime  passenger_count  pickup_longitude  \
0  id2793718 2016-06-08 07:36:19                1        -73.985611   
1  id3485529 2016-04-03 12:58:11                1        -73.978394   
2  id1816614 2016-06-05 02:49:13                5        -73.989059   
3  id1050851 2016-05-05 17:18:27                2        -73.990326   
4  id0140657 2016-05-12 17:43:38                4        -73.789497   
5  id2734699 2016-04-14 12:17:42                3        -73.969833   

 EDA نحفظ بقا الملف عشان نرجع نبص عالعلاقات فالنةتبةك بتاع ال 

In [98]:
save_processed_data(df, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv")


✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv


نرجع نبص بصه 

In [99]:
path = "G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv"
df_processed = pd.read_csv(path)
df_processed['pickup_datetime'] = pd.to_datetime(df_processed['pickup_datetime'])

df_processed.head()
print(df_processed.shape)
print(df_processed.info())
df_processed.describe()

(1000000, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   id                     1000000 non-null  object        
 1   pickup_datetime        1000000 non-null  datetime64[ns]
 2   passenger_count        1000000 non-null  int64         
 3   pickup_longitude       1000000 non-null  float64       
 4   pickup_latitude        1000000 non-null  float64       
 5   dropoff_longitude      1000000 non-null  float64       
 6   dropoff_latitude       1000000 non-null  float64       
 7   trip_duration_min      1000000 non-null  float64       
 8   haversine_km           1000000 non-null  float64       
 9   manhattan_km           1000000 non-null  float64       
 10  bearing_deg            1000000 non-null  float64       
 11  speed_kmh              1000000 non-null  float64       
 12  hour           

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration_min,haversine_km,manhattan_km,bearing_deg,speed_kmh,hour,is_weekend,is_rush_hour,log_trip_duration_min
count,1000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,2016-04-01 08:53:15.180106752,1.665353,-73.973475,40.750947,-73.973421,40.751829,15.914750,3.439099,4.444066,-15.768007,14.428394,13.604165,0.285775,0.428937,2.488104
min,2016-01-01 00:00:53,0.000000,-121.933342,34.359695,-121.933304,34.359695,0.016667,0.000000,0.000000,-179.992701,0.000000,0.000000,0.000000,0.000000,0.016529
25%,2016-02-17 15:26:08,1.000000,-73.991852,40.737372,-73.991341,40.735928,6.616667,1.232068,1.570859,-125.293788,9.118796,9.000000,0.000000,0.000000,2.030339
50%,2016-04-01 15:01:29.500000,1.000000,-73.981728,40.754131,-73.979767,40.754551,11.033333,2.091231,2.685980,8.043926,12.787544,14.000000,0.000000,0.000000,2.487681
75%,2016-05-15 02:35:38.750000128,2.000000,-73.967346,40.768379,-73.963036,40.769833,17.900000,3.871264,4.993929,53.263991,17.842133,19.000000,1.000000,1.000000,2.939162
max,2016-06-30 23:59:37,7.000000,-61.335529,51.881084,-61.335529,43.921028,37126.866667,1240.908677,1336.846365,180.000000,9274.836731,23.000000,1.000000,1.000000,10.522123
std,NaN,1.315723,0.065404,0.033745,0.065432,0.035782,64.701169,4.424258,5.806581,104.460439,16.575388,6.400685,0.451783,0.494925,0.694773


لاحظت ان في قيم مش منطقيه زي ان في مدة رحله وصل ل 40 يوم فغالبا في اوتلايرز محتاجين نشوفها ونتعامل معاها


In [100]:
columns = ['trip_duration_min', 'haversine_km', 'manhattan_km', 'speed_kmh']

factor = 2  # ممكن تزوده لـ 2.5 لو عايز تقلل أكتر

for col in columns:
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    outliers = df_processed[(df_processed[col] < lower) | (df_processed[col] > upper)]
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_processed)*100:.2f}%)")

trip_duration_min: 32602 outliers (3.26%)
haversine_km: 75358 outliers (7.54%)
manhattan_km: 66786 outliers (6.68%)
speed_kmh: 22601 outliers (2.26%)


دي كده نسبة الامثله اللي فيها اوتلاير واضح 
من النسبه دي من رايي نسبه مش كبيره االافضل نمسحها

In [101]:
columns = ['trip_duration_min', 'haversine_km', 'manhattan_km', 'speed_kmh']

factor = 2  # تقدر تغيره لتقليل أو زيادة عدد القيم الشاذة

df_clean = df_processed.copy()
original_rows = len(df_clean)

for col in columns:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

cleaned_rows = len(df_clean)
deleted_rows = original_rows - cleaned_rows
deleted_percent = deleted_rows / original_rows * 100



print(f"Original rows: {original_rows}")
print(f"Cleaned rows: {cleaned_rows}")
print(f"Deleted rows: {deleted_rows} ({deleted_percent:.2f}%)")

Original rows: 1000000
Cleaned rows: 873396
Deleted rows: 126604 (12.66%)


In [102]:
df_clean.head()
print(df_clean.shape)
print(df_clean.info())
df_clean.describe()

(873396, 33)
<class 'pandas.core.frame.DataFrame'>
Index: 873396 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     873396 non-null  object        
 1   pickup_datetime        873396 non-null  datetime64[ns]
 2   passenger_count        873396 non-null  int64         
 3   pickup_longitude       873396 non-null  float64       
 4   pickup_latitude        873396 non-null  float64       
 5   dropoff_longitude      873396 non-null  float64       
 6   dropoff_latitude       873396 non-null  float64       
 7   trip_duration_min      873396 non-null  float64       
 8   haversine_km           873396 non-null  float64       
 9   manhattan_km           873396 non-null  float64       
 10  bearing_deg            873396 non-null  float64       
 11  speed_kmh              873396 non-null  float64       
 12  hour                   873396 non-nu

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration_min,haversine_km,manhattan_km,bearing_deg,speed_kmh,hour,is_weekend,is_rush_hour,log_trip_duration_min
count,873396,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000,873396.000000
mean,2016-04-01 01:27:25.571159040,1.661807,-73.979676,40.752901,-73.978030,40.752866,11.522303,2.290994,2.940740,-16.525115,12.767369,13.715420,0.285035,0.440354,2.369378
min,2016-01-01 00:00:53,0.000000,-121.933342,34.359695,-121.933304,34.359695,0.016667,0.000000,0.000000,-179.991787,0.000000,0.000000,0.000000,0.000000,0.016529
25%,2016-02-17 09:13:31.249999872,1.000000,-73.992172,40.738670,-73.991699,40.738014,6.250000,1.152322,1.465313,-130.006434,8.723518,9.000000,0.000000,0.000000,1.981001
50%,2016-04-01 01:13:11.500000,1.000000,-73.982224,40.754246,-73.980942,40.754616,10.033333,1.861010,2.386704,7.435483,11.976896,14.000000,0.000000,0.000000,2.400921
75%,2016-05-14 18:58:39.750000128,2.000000,-73.969833,40.767700,-73.967133,40.768742,15.283333,3.033323,3.909568,52.079819,16.068112,19.000000,1.000000,1.000000,2.790142
max,2016-06-30 23:59:37,7.000000,-61.335529,43.911762,-61.335529,43.911762,40.450000,8.414595,9.276746,180.000000,31.431036,23.000000,1.000000,1.000000,3.724488
std,NaN,1.314189,0.060098,0.026965,0.060545,0.028992,6.996582,1.542302,1.983405,105.290701,5.679450,6.294113,0.451431,0.496430,0.582552


In [103]:
save_processed_data(df_clean, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed_filtered.csv")


✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed_filtered.csv


كده خلصنا جزء التنضيف والفيتشر انجنيرنج للترين داتا ناقص نظبط شكل الفال والتيست زي الترين ولكن من غير تغيير قيم او حذف صفوف 
كل اللي هنعمله عليهم هو إضافة نفس الـ features اللي عملناها في train:

أي encoding للـ categorical variables.

أي columns جديدة (feature engineering).

scaling / normalization بنفس المعايير اللي استخدمناها في train.
السبب: عشان الموديل يتقيم على بيانات لم يشوفها قبل كده، ونحافظ على consistency بين train و validation/test.

In [107]:
df_val = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\val.csv")
df_test = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\test.csv")

df_val['pickup_datetime'] = pd.to_datetime(df_val['pickup_datetime'])
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

print(df_val.shape)
print(df_val.head())
print(df_val.info())

print(df_test.shape)
print(df_test.head())
print(df_test.info())

(229319, 10)
          id  vendor_id     pickup_datetime  passenger_count  \
0  id2542907          1 2016-01-10 16:01:46                1   
1  id1709406          1 2016-06-23 18:41:05                1   
2  id0983698          1 2016-05-14 21:25:34                1   
3  id3045153          2 2016-05-02 20:09:00                1   
4  id2383875          1 2016-05-19 10:01:39                1   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -74.013916        40.713444         -73.993858         40.752510   
1        -74.005440        40.727306         -73.983063         40.734715   
2        -73.987587        40.749863         -73.986809         40.757549   
3        -73.973862        40.784153         -73.983025         40.774479   
4        -73.999916        40.733101         -74.008331         40.734177   

  store_and_fwd_flag  trip_duration  
0                  N           1249  
1                  N            817  
2                  N     

In [108]:
# 1️⃣ إضافة الـ distance features

df_val = add_distance_features(df_val)
df_test = add_distance_features(df_test)

# 2️⃣ إضافة الـ time-based features
df_val = add_time_features(df_val)
df_test = add_time_features(df_test)


df_val['trip_duration_min']   = df_val['trip_duration'] / 60
df_test['trip_duration_min']  = df_test['trip_duration'] / 60

df_val = df_val.drop(columns=["trip_duration"])
df_test = df_test.drop(columns=["trip_duration"])

# 3️⃣ إضافة الـ one-hot encoding بنفس الأعمدة اللي في train
df_val = add_one_hot_encoding(df_val)
df_test = add_one_hot_encoding(df_test)

df_val['log_trip_duration_min']   = np.log1p(df_val['trip_duration_min'])
df_test['log_trip_duration_min']  = np.log1p(df_test['trip_duration_min'])



✅ add_distance_features: 229319/229319 rows got valid speed_kmh
✅ add_distance_features: 229322/229322 rows got valid speed_kmh


In [109]:
print(df_val.shape)
print(df_val.head())
print(df_val.info())

print(df_val.columns.tolist())

print(df_test.shape)
print(df_test.head())
print(df_test.info())

(229319, 33)
          id     pickup_datetime  passenger_count  pickup_longitude  \
0  id2542907 2016-01-10 16:01:46                1        -74.013916   
1  id1709406 2016-06-23 18:41:05                1        -74.005440   
2  id0983698 2016-05-14 21:25:34                1        -73.987587   
3  id3045153 2016-05-02 20:09:00                1        -73.973862   
4  id2383875 2016-05-19 10:01:39                1        -73.999916   

   pickup_latitude  dropoff_longitude  dropoff_latitude  haversine_km  \
0        40.713444         -73.993858         40.752510      4.661154   
1        40.727306         -73.983063         40.734715      2.057606   
2        40.749863         -73.986809         40.757549      0.857222   
3        40.784153         -73.983025         40.774479      1.323778   
4        40.733101         -74.008331         40.734177      0.719070   

   manhattan_km  bearing_deg  ...  weekday_4  weekday_5  weekday_6  month_1  \
0      6.033519    21.252151  ...      Fal

دلوقتي نفظ الملفات دي بتاع الفال والتيست بعد ما خليناهم متشابهين مع الترين فالاعمده بدون التدخل فقيم الاعمده نفسها

In [110]:
save_processed_data(df_val, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/val_processed.csv")
save_processed_data(df_test, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/test_processed.csv")

✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/val_processed.csv
✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/test_processed.csv
